In [1]:
import xgboost as xgb
print(xgb.__version__)

3.0.4


In [2]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

/home/basil-owiti/ML_RegPipeline_Endtoend/.venv/bin/python
3.0.4
/home/basil-owiti/ML_RegPipeline_Endtoend/.venv/lib/python3.11/site-packages/xgboost/__init__.py


In [4]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost
from here import here
import os


home_dir = os.path.dirname(here())
data_dir = os.path.join(home_dir, "data")
data_dir

plot_dir = os.path.join(home_dir, "plots")

In [5]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv(os.path.join(data_dir, "processed", "feature_engineered_train.csv"))
eval_df  = pd.read_csv(os.path.join(data_dir, "processed", "feature_engineered_eval.csv"))


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (576815, 39)
Eval shape: (148448, 39)


In [ ]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    """pass possible parameter values to optuna for experimentation"""
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }
    # track runs with mlflow
    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [9]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder
mlrun_dir = os.path.join(home_dir, "mlruns")
mlflow.set_tracking_uri(mlrun_dir)
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

/home/basil-owiti/ML_RegPipeline_Endtoend/.venv/lib/python3.11/site-packages/mlflow/tracking/_tracking_service/utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
2025/12/27 18:30:06 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.
[I 2025-12-27 18:30:06,153] A new study created in memory with name: no-name-ebc675df-042f-44fc-8796-e52a180d0494
[I 2025-12-27 18:32:29,639] Trial 0 finished with value: 68119.8893549254 and parameters: {'n_estimators': 236, 'max_depth': 8, 'learning_rate': 0.057427200730873144, 'subsamp

Best params: {'n_estimators': 236, 'max_depth': 8, 'learning_rate': 0.057427200730873144, 'subsample': 0.9665654781256106, 'colsample_bytree': 0.6357966360557227, 'min_child_weight': 4, 'gamma': 3.393559749900545, 'reg_alpha': 5.217343314898461e-07, 'reg_lambda': 0.004241980251497593}


In [11]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
# Add this line right before your mlflow.start_run()
best_model._estimator_type = "regressor"
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 32242.70747991763
RMSE: 70579.9689058883
R²: 0.961503348407537


2025/12/27 19:56:50 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmplprw461u/model, flavor: xgboost). Fall back to return ['xgboost==3.0.4']. Set logging level to DEBUG to see the full traceback. 
2025/12/27 19:56:50 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
